### Importing Important Libraries

#### Steps To Be Followed
1. Importing necessary Libraries
2. Creating S3 bucket 
3. Mapping train And Test Data in S3
4. Mapping The path of the models in S3

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import image_uris
from sagemaker.session import s3_input, Session

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
bucket_name = 'banksagemaker' 
my_region = boto3.session.Session().region_name 
print(my_region)

us-east-1


In [3]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://banksagemaker/xgboost-as-a-built-in-algo/output


#### Downloading The Dataset And Storing in S3

In [4]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [5]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [7]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

### Building Models Xgboot- Inbuilt Algorithm

In [8]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
container = image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

In [9]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [10]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [11]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: xgboost-2024-01-14-10-25-10-316


2024-01-14 10:25:10 Starting - Starting the training job...
2024-01-14 10:25:24 Starting - Preparing the instances for training......
2024-01-14 10:26:26 Downloading - Downloading input data...
2024-01-14 10:26:51 Downloading - Downloading the training image...
2024-01-14 10:27:17 Training - Training image download completed. Training in progress.Arguments: train
[2024-01-14:10:27:35:INFO] Running standalone xgboost training.
[2024-01-14:10:27:35:INFO] File size need to be processed in the node: 4.83mb. Available memory size in the node: 23945.05mb
[2024-01-14:10:27:35:INFO] Determined delimiter of CSV input is ','
[10:27:35] S3DistributionType set as FullyReplicated
[10:27:35] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-01-14:10:27:35:INFO] Determined delimiter of CSV input is ','
[10:27:35] S3DistributionType set as FullyReplicated
[10:27:35] 12357x59 matrix with 729063 entries loaded from /opt/ml/input/data/va

### Deploy Machine Learning Model As Endpoints

In [12]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2024-01-14-10-28-22-605
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-01-14-10-28-22-605
INFO:sagemaker:Creating endpoint with name xgboost-2024-01-14-10-28-22-605


-----!

#### Prediction of the Test Data

In [13]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [14]:
predictions_array

array([0.05578489, 0.04585116, 0.05088526, ..., 0.03450042, 0.03321051,
       0.03635123])

In [15]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10784)    36% (152)
Purchase        10% (1149)     64% (272) 



#### Deleting The Endpoints

In [18]:
sagemaker.predictor.Predictor.delete_endpoint(xgb_predictor) 
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-2024-01-14-10-28-22-605
INFO:sagemaker:Deleting endpoint with name: xgboost-2024-01-14-10-28-22-605


[{'ResponseMetadata': {'RequestId': '3ZS7WP9ZBR9HG504',
   'HostId': 'Kv4tqKJJBcvw7nwJ/vkbTrmyB81VMH5xPurGPxHdzP3/1ajTXFWsl3AS9+yV/jv1PgmlJpkqcKY=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'Kv4tqKJJBcvw7nwJ/vkbTrmyB81VMH5xPurGPxHdzP3/1ajTXFWsl3AS9+yV/jv1PgmlJpkqcKY=',
    'x-amz-request-id': '3ZS7WP9ZBR9HG504',
    'date': 'Sun, 14 Jan 2024 10:33:28 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2024-01-14-10-25-10-316/profiler-output/system/incremental/2024011410/1705228020.algo-1.json'},
   {'Key': 'xgboost-as-a-built-in-algo/train/train.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/test/test.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2024-01-14-10-25-10-316/debug-output/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2024-01-14-10-25-10-316/pro